In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import webbrowser
import folium
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import time

In [8]:
#importing the data
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df = pd.read_csv('data/real_estate/synthetic_data.csv')
df.head(10)


,brokered_by,price,bed,bath,acre_lot,city,state,zip_code,house_size,year,month,unemployment_yr_change,cpi_yr_change,real_gdp_yr_change
0,50816.0,59010.065151,4.0,4.0,0.24,Canton,Georgia,30115.0,3305.0,1975,4,3.8,4.9,-1.8
1,30807.0,14854.018749,3.0,1.0,0.40,Salem,New Jersey,8079.0,864.0,1975,4,3.8,4.9,-1.8
2,26201.0,73218.141378,3.0,4.0,0.05,Dallas,Texas,75219.0,2807.0,1975,1,3.3,5.5,-2.3
3,19178.0,18177.294285,2.0,1.0,19.72,Steubenville,Ohio,43952.0,805.0,1975,3,3.8,4.9,-2.3
4,35637.0,69798.320534,3.0,2.0,2.50,Naples,Florida,34116.0,1450.0,1975,11,1.6,3.8,2.6
5,58988.0,70112.420812,5.0,3.0,3.06,Finksburg,Maryland,21048.0,3501.0,1975,3,3.8,4.9,-2.3
6,46344.0,92133.783105,2.0,3.0,7.75,Aiken,South Carolina,29803.0,1560.0,1975,11,1.6,3.8,2.6
7,66304.0,18810.139076,3.0,2.0,1.70,Bedford,Virginia,24523.0,1880.0,1975,4,3.8,4.9,-1.8
8,22654.0,26995.878769,3.0,1.0,1.30,Richmond,Rhode Island,2898.0,1224.0,1975,3,3.8,4.9,-2.3
9,22611.0,8725.101257,3.0,2.0,0.08,Chicago,Illinois,60621.0,2644.0,1975,3,3.8,4.9,-2.3


Now we want to iterate over every row of the city, state and zip-code to find the coordinates and append the df with a new "coordinates" variable

In [ ]:
# Assume df is your original DataFrame.
# For example, df.columns includes: ['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot',
# 'street', 'city', 'state', 'zip_code', 'house_size', 'prev_sold_date', ...]

# Check if the original DataFrame has at least 1200 rows
if len(df) < 1200:
    raise ValueError(f"Original dataset has only {len(df)} rows; cannot sample 1200 unique rows without duplicates.")

# Determine the number of unique states
n_states = df['state'].nunique()
# Calculate desired number per state (integer division)
n_per_state = 1200 // n_states

# Stratify by state: For each state, sample up to n_per_state rows without replacement.
# (If a state has fewer than n_per_state rows, take all available rows.)
df_stratified = df.groupby('state', group_keys=False).apply(
    lambda x: x.sample(n=min(n_per_state, len(x)), random_state=42)
).reset_index(drop=True)

# If the stratified sample has fewer than 1200 rows, then attempt to get additional unique rows.
if len(df_stratified) < 1200:
    extra_needed = 1200 - len(df_stratified)
    # Get the remaining rows that were not selected in the stratified sample.
    remaining = df.loc[~df.index.isin(df_stratified.index)]
    
    if len(remaining) < extra_needed:
        # If there aren't enough remaining rows, print a warning.
        print(f"Warning: Only {len(remaining)} remaining rows are available, so you'll have fewer than 1200 unique rows.")
        extra_rows = remaining  # Use all remaining rows.
    else:
        extra_rows = remaining.sample(n=extra_needed, random_state=42, replace=False)
    
    df_sample = pd.concat([df_stratified, extra_rows]).reset_index(drop=True)
else:
    df_sample = df_stratified

print(df_sample[['city', 'state', 'zip_code']])
print("Number of rows in Sample df:", len(df_sample))


              city           state  zip_code
0           Pinson         Alabama   35126.0
1       Fultondale         Alabama   35068.0
2       Birmingham         Alabama   35242.0
3       Birmingham         Alabama   35216.0
4           Pelham         Alabama   35124.0
...            ...             ...       ...
1195     Charlotte  North Carolina   28209.0
1196       Houston           Texas   77084.0
1197  East Concord        New York   14055.0
1198        Sparks          Nevada   89436.0
1199      Lakewood        Illinois   60014.0

[1200 rows x 3 columns]
Number of rows in Sample df: 1200


/tmp/ipykernel_8755/2730975796.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_stratified = df.groupby('state', group_keys=False).apply(


In [10]:
print(df_sample)

      brokered_by          price  bed  bath  acre_lot          city           state  zip_code  house_size  year  month  unemployment_yr_change  cpi_yr_change  real_gdp_yr_change
0         80881.0   59895.804924  4.0   3.0      0.48        Pinson         Alabama   35126.0      2608.0  1986     11                    -0.1          1.400                 2.9
1         56101.0   14962.459628  2.0   3.0      0.47    Fultondale         Alabama   35068.0      1435.0  1979     11                     0.0          8.500                 1.3
2         53423.0  145233.063316  2.0   3.0      0.02    Birmingham         Alabama   35242.0      1486.0  2013      1                    -0.3          3.615                 1.7
3         84178.0   66427.846436  2.0   2.0      0.02    Birmingham         Alabama   35216.0       944.0  2001      3                     0.2          5.000                 2.2
4         84184.0   65401.277543  3.0   2.0      0.58        Pelham         Alabama   35124.0      2093.0  198

Specific test

In [12]:
import pandas as pd
import ssl
import certifi
from geopy.geocoders import ArcGIS
import os
import concurrent.futures
from tqdm import tqdm
import time

# Set the SSL_CERT_FILE environment variable so ArcGIS can verify certificates
os.environ["SSL_CERT_FILE"] = certifi.where()

# Assume df_sample is already defined and has columns: 'city', 'state', 'zip_code'
# Ensure a full_address column exists in df_sample:
df_sample['full_address'] = (
    df_sample['city'] + ', ' +
    df_sample['state'] + ', ' +
    df_sample['zip_code'].astype(str) + ', United States'
)

# Initialize the ArcGIS geocoder
geolocator = ArcGIS()

def geocode_address(addr):
    """Geocode a single address using ArcGIS with a timeout and a short delay."""
    try:
        location = geolocator.geocode(addr, timeout=10)
        time.sleep(0.1)  # Reduced delay between requests; adjust as needed
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding {addr}: {e}")
        return (None, None)

def process_address(addr):
    """Wrapper function for geocoding an address."""
    return geocode_address(addr)

# Get list of full addresses from the sample
addresses = df_sample['full_address'].tolist()

# Use ThreadPoolExecutor for parallel processing
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Map process_address function to each address in parallel
    results = list(tqdm(executor.map(process_address, addresses), total=len(addresses)))

# Unzip the results into separate lists for latitude and longitude
latitudes, longitudes = zip(*results)
df_sample['latitude'] = latitudes
df_sample['longitude'] = longitudes

print("Data with geocoded coordinates using ArcGIS:")
print(df_sample[['city', 'state', 'zip_code', 'full_address', 'latitude', 'longitude']])


100%|██████████| 1200/1200 [04:35<00:00,  4.35it/s]

Data with geocoded coordinates using ArcGIS:
              city           state  zip_code                                       full_address   latitude   longitude
0           Pinson         Alabama   35126.0            Pinson, Alabama, 35126.0, United States  33.679170  -86.680982
1       Fultondale         Alabama   35068.0        Fultondale, Alabama, 35068.0, United States  33.604243  -86.797908
2       Birmingham         Alabama   35242.0        Birmingham, Alabama, 35242.0, United States  33.417226  -86.683463
3       Birmingham         Alabama   35216.0        Birmingham, Alabama, 35216.0, United States  33.407767  -86.805955
4           Pelham         Alabama   35124.0            Pelham, Alabama, 35124.0, United States  33.312978  -86.801347
...            ...             ...       ...                                                ...        ...         ...
1195     Charlotte  North Carolina   28209.0  Charlotte, North Carolina, 28209.0, United States  35.539344  -79.185418
119

Great now I will update the amenities code across the US

In [13]:
import pandas as pd
import requests
import time
import geopandas as gpd
from shapely.geometry import Point
import concurrent.futures
from tqdm import tqdm

# Assume df_sample is already defined and includes:
# 'city', 'state', 'zip_code', 'full_address', 'latitude', 'longitude'
df_sample['full_address'] = (
    df_sample['city'] + ', ' +
    df_sample['state'] + ', ' +
    df_sample['zip_code'].astype(str) + ', United States'
)

def get_circle_bbox(lat, lon, radius_m=500):
    """Create a circular buffer around the point and return its bounding box."""
    point = Point(lon, lat)  # Note: Point takes (lon, lat)
    gdf = gpd.GeoDataFrame({'geometry': [point]}, crs='EPSG:4326')
    gdf_metric = gdf.to_crs(epsg=3857)
    gdf_metric['geometry'] = gdf_metric.buffer(radius_m)
    poly = gdf_metric.to_crs(epsg=4326)['geometry'].iloc[0]
    return poly.bounds  # (minx, miny, maxx, maxy)

def query_overpass(bbox, retries=3, delay=3):
    """
    Query Overpass API for OSM elements with an "amenity" tag within the bounding box.
    Retries the request up to 'retries' times if it fails to get a valid JSON response.
    """
    minx, miny, maxx, maxy = bbox
    query = f"""
    [out:json][timeout:25];
    (
      node["amenity"]({miny},{minx},{maxy},{maxx});
      way["amenity"]({miny},{minx},{maxy},{maxx});
      relation["amenity"]({miny},{minx},{maxy},{maxx});
    );
    out center;
    """
    # You can try an alternate Overpass API endpoint if needed:
    url = "http://overpass-api.de/api/interpreter"
    # Alternatively, try:
    # url = "https://lz4.overpass-api.de/api/interpreter"
    
    for attempt in range(1, retries+1):
        try:
            response = requests.get(url, params={'data': query})
            if not response.text.strip():
                raise ValueError("Empty response")
            data = response.json()
            return data
        except Exception as e:
            print(f"Overpass query error (attempt {attempt}/{retries}): {e}")
            time.sleep(delay)
    return {}  # Return an empty dict if all retries fail.

def count_overpass_amenities(lat, lon, radius_m=500):
    """Query Overpass API for a point's buffered area and return the count of amenities."""
    try:
        bbox = get_circle_bbox(lat, lon, radius_m)
        data = query_overpass(bbox, retries=3, delay=3)
        time.sleep(0.5)  # Short delay between requests
        if 'elements' in data:
            return len(data['elements'])
    except Exception as e:
        print(f"Error querying amenities for point ({lat}, {lon}): {e}")
    return 0

def process_row(row):
    return count_overpass_amenities(row.latitude, row.longitude, 500)

results = []
try:
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        results = list(tqdm(executor.map(process_row, df_sample.itertuples(index=False)), total=len(df_sample)))
except KeyboardInterrupt:
    print("Processing was interrupted by the user.")

if results:
    df_sample['amenity_count_500m'] = results
    print(df_sample[['city', 'state', 'zip_code', 'full_address', 'amenity_count_500m']])
else:
    print("No results obtained.")


 70%|███████   | 843/1200 [06:33<07:14,  1.22s/it]

Overpass query error (attempt 1/3): Expecting value: line 1 column 1 (char 0)
Overpass query error (attempt 1/3): Expecting value: line 1 column 1 (char 0)


 70%|███████   | 845/1200 [06:43<11:30,  1.94s/it]

Overpass query error (attempt 1/3): Expecting value: line 1 column 1 (char 0)
Overpass query error (attempt 1/3): Expecting value: line 1 column 1 (char 0)


 72%|███████▏  | 867/1200 [07:12<07:32,  1.36s/it]

Overpass query error (attempt 1/3): Expecting value: line 1 column 1 (char 0)
Overpass query error (attempt 1/3): Expecting value: line 1 column 1 (char 0)
Overpass query error (attempt 1/3): Expecting value: line 1 column 1 (char 0)


100%|██████████| 1200/1200 [11:12<00:00,  1.78it/s]

              city           state  zip_code                                       full_address  amenity_count_500m
0           Pinson         Alabama   35126.0            Pinson, Alabama, 35126.0, United States                   0
1       Fultondale         Alabama   35068.0        Fultondale, Alabama, 35068.0, United States                   3
2       Birmingham         Alabama   35242.0        Birmingham, Alabama, 35242.0, United States                  34
3       Birmingham         Alabama   35216.0        Birmingham, Alabama, 35216.0, United States                  29
4           Pelham         Alabama   35124.0            Pelham, Alabama, 35124.0, United States                   0
...            ...             ...       ...                                                ...                 ...
1195     Charlotte  North Carolina   28209.0  Charlotte, North Carolina, 28209.0, United States                   0
1196       Houston           Texas   77084.0             Houston, Texas,

In [14]:
print(df_sample)

      brokered_by          price  bed  bath  acre_lot          city           state  zip_code  house_size  year  month  unemployment_yr_change  cpi_yr_change  real_gdp_yr_change                                       full_address   latitude   longitude  amenity_count_500m
0         80881.0   59895.804924  4.0   3.0      0.48        Pinson         Alabama   35126.0      2608.0  1986     11                    -0.1          1.400                 2.9            Pinson, Alabama, 35126.0, United States  33.679170  -86.680982                   0
1         56101.0   14962.459628  2.0   3.0      0.47    Fultondale         Alabama   35068.0      1435.0  1979     11                     0.0          8.500                 1.3        Fultondale, Alabama, 35068.0, United States  33.604243  -86.797908                   3
2         53423.0  145233.063316  2.0   3.0      0.02    Birmingham         Alabama   35242.0      1486.0  2013      1                    -0.3          3.615                 1.7       

In [18]:
df_sample.drop(columns='full_address', inplace=True)

In [19]:
# Count the number of NaN values in the column
nan_count = df_sample['amenity_count_500m'].isna().sum()
print("Number of NaN values:", nan_count)

Number of NaN values: 0


In [ ]:
df_sample.to_csv("data/real_estate/synthetic_data.csv", index=False)